In [1]:
import findspark
findspark.init()

import pyspark
from pyspark.sql import SparkSession

In [2]:
def create_spark_session():
    spark = SparkSession \
      .builder \
      .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:3.2.0") \
      .config("spark.hadoop.fs.s3a.impl","org.apache.hadoop.fs.s3a.S3AFileSystem") \
      .getOrCreate()
  
    sc=spark.sparkContext
    return spark

In [3]:
spark = create_spark_session()

In [4]:
spark

In [6]:
sc=spark.sparkContext
hadoop_conf = sc._jsc.hadoopConfiguration()

hadoop_conf.set("fs.s3a.endpoint", '127.0.0.1:9000')
hadoop_conf.set("fs.s3a.access.key", 'minioadmin')
hadoop_conf.set("fs.s3a.secret.key",'minioadmin')
hadoop_conf.set("fs.s3a.connection.ssl.enabled", "false") 
hadoop_conf.set("spark.hadoop.fs.s3a.path.style.access", "true")
hadoop_conf.set("fs.s3a.aws.credentials.provider", "org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider")
hadoop_conf.set("fs.s3a.impl","org.apache.hadoop.fs.s3a.S3AFileSystem")

##### we’re going to read a directory of JSON files and enforce a schema on load to make sure each file has all of the columns that we’re expecting.

In [7]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType,LongType, BooleanType,TimestampType

In [8]:
data_schema = [StructField('message',StructType([StructField('id', LongType(), True),
                           StructField('id_str', StringType(), True),
                           StructField('name', StringType(), True),
                           StructField('screen_name', StringType(), True),
                           StructField('location', StringType(), True),
                           StructField('description', StringType(), True),
                           StructField('url', StringType(), True),
                           StructField('protected', BooleanType(), True),
                           StructField('followers_count', LongType(), True),
                           StructField('friends_count', LongType(), True),
                           StructField('listed_count', LongType(), True),
                           StructField('created_at', StringType(), True),
                           StructField('favourites_count', StringType(), True),
                           StructField('statuses_count', LongType(), True),
                           StructField('lang', StringType(), True),
                           StructField('profile_image_url_https', StringType(), True)])),
              StructField('timestamp', LongType(), True)]

final_struct = StructType(fields=data_schema)

## 1- Read JSON files from Minio.
## and at the same time
## 2 - Select the following fields; id, id_str, name, screen_name, location, description, url, protected, followers_count, friends_count, listed_count, created_at, favourites_count, statuses_count, lang, profile_image_url_https, timestamp.

## we can do it in this way because spark is Schema On Read

In [10]:
df = spark.read.json("s3a://users/*.json",schema=final_struct)

In [12]:
df.show(2)

+--------------------+-------------+
|             message|    timestamp|
+--------------------+-------------+
|{1005019322991955...|1579706091269|
|{16341288, 163412...|1579706091271|
+--------------------+-------------+
only showing top 2 rows



#### Since we have to use  SPARK SQL , we shoud create view which allow us to use sql .

In [13]:
df.createOrReplaceTempView('userdata')

In [14]:
df2 = spark.sql('''select 
message.*,
timestamp
from userdata

''')

In [15]:
df2.createOrReplaceTempView('userdata2')

# 3 - Remove duplicate users.

### i picked last user recored based on timestamp.

In [17]:
df3 = spark.sql('''
select U.* from userdata2 U
inner join
(select id,timestamp, row_number() over (partition by id order by timestamp desc ) as recordrank from userdata2) H
on U.id=H.id and U.timestamp = H.timestamp
where H.recordrank=1
''')

In [18]:
df3.createOrReplaceTempView('userdata3')

# 4- Remove space characters from description, name, location, and URL fields.
# 5- Convert created_at field to DateTime with (year-month-day) format.

In [19]:
df4 = spark.sql('''
select 
id,
id_str,
replace(name,' ','') AS name,
screen_name,
replace(location,' ','') AS location,
replace(description,' ','') AS description,
replace(url,' ','') AS url,
protected,
followers_count,
friends_count,
listed_count,
date_format(to_timestamp(substr(created_at,5),'MMM dd HH:mm:ss Z yyyy'),'yyyy-MM-dd') AS created_at,
favourites_count,
statuses_count,
lang,
profile_image_url_https,
timestamp
from userdata3

''')

# 6- Load data in MINIO.
##### because i want to have just one csv file i do repartitioning

In [21]:
df4.repartition(1).write.csv('s3a://users/users.csv',header=True)